In [1]:
import pandas as pd 
from sklearn.linear_model import LogisticRegression 
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score



In [37]:
csv_list = os.listdir("data/nba_games_2021/")
cols1 = {'away_winpct':[], 'home_winpct':[], 'home_win':[]}
pregame_data = pd.DataFrame(cols1)
cols2 = {'away_pts':[], 'home_pts':[], 'seconds_rem':[], 'home_win':[]}
ingame_data = pd.DataFrame(cols2)
for csv in csv_list:
    #print(csv)
    if (csv == "urls2021"): continue
    game = pd.read_csv(f'data/nba_games_2021/{csv}')
    home = game.columns[1]
    away = game.columns[0]
    home_record = game.loc[0][1]
    away_record = game.loc[0][0]
    if (type(home_record)!=type("hi") or type(away_record)!=type("hi")): continue
    home_winpct = int(home_record.split("-")[0]) / (int(home_record.split("-")[0])+int(home_record.split("-")[1]))
    away_winpct = int(away_record.split("-")[0]) / (int(away_record.split("-")[0])+int(away_record.split("-")[1]))   
    if (game.iloc[-2][0].startswith("J")):
        final_score = final_score = (int(game.iloc[-3][1]), int(game.iloc[-3][0]))
    else: final_score = (int(game.iloc[-2][1]), int(game.iloc[-2][0]))
    home_win = 1 if final_score[0] > final_score[1] else 0
    row = [away_winpct, home_winpct, home_win]
    pregame_data.loc[len(pregame_data.index)] = row

    game = game.drop(labels = 0, axis = 0)
    def time_to_seconds(time_str):
        minutes, seconds = 0, 0
        if 'M' in time_str:
            splitter = time_str.split('M')
            minutes = int(splitter[0])
            seconds = int(splitter[1].split('S')[0])
        else:
            seconds = int(time_str.split('S')[0])
        return minutes * 60 + seconds
    game['seconds_rem'] = game['time_rem'].apply(time_to_seconds)
    game = game.drop(columns = ['time_rem'])
    game = game.drop(labels = len(game), axis = 0)
    game['home_win'] = home_win
    game = game.rename(columns ={away:'away_pts', home:'home_pts'})
    game = game[~game['away_pts'].str.startswith(("S", "E", "J"))]
    game['away_pts'] = game['away_pts'].astype(int)
    game['home_pts'] = game['home_pts'].astype(int)
    ingame_data = ingame_data.append(game)
    #print(ingame_data)


/var/folders/cl/6y9p0b1j7_l0kfxst3x5zbc40000gn/T/ipykernel_85988/3240328539.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ingame_data = ingame_data.append(game)
/var/folders/cl/6y9p0b1j7_l0kfxst3x5zbc40000gn/T/ipykernel_85988/3240328539.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ingame_data = ingame_data.append(game)
/var/folders/cl/6y9p0b1j7_l0kfxst3x5zbc40000gn/T/ipykernel_85988/3240328539.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ingame_data = ingame_data.append(game)
/var/folders/cl/6y9p0b1j7_l0kfxst3x5zbc40000gn/T/ipykernel_85988/3240328539.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ingame_data

In [33]:
#Pregame
predictors = pregame_data.drop(columns=["home_win"])
response = pregame_data["home_win"]
train_data, test_data, train_target, test_target = train_test_split(
    predictors, response, test_size=0.8, random_state=999)

model = LogisticRegression() 
model.fit(train_data, train_target) 
pregame_win_probabilities = model.predict_proba(test_data)
print(pregame_win_probabilities)


[[0.43777767 0.56222233]
 [0.25959348 0.74040652]
 [0.54325485 0.45674515]
 ...
 [0.64304203 0.35695797]
 [0.43785354 0.56214646]
 [0.4445252  0.5554748 ]]


In [39]:
#Ingame
pred = ingame_data.drop(columns=["home_win"])
resp = ingame_data["home_win"]
#print(ingame_data)
x_train, x_test, y_train, y_test = train_test_split(
    pred, resp, test_size=0.8, random_state=999)

ingame_model = LogisticRegression() 
ingame_model.fit(x_train, y_train) 
ingame_win_probabilities = model.predict_proba(test_data)
ingame_win_probabilities


array([[0.43777767, 0.56222233],
       [0.25959348, 0.74040652],
       [0.54325485, 0.45674515],
       ...,
       [0.64304203, 0.35695797],
       [0.43785354, 0.56214646],
       [0.4445252 , 0.5554748 ]])

In [ ]:
# Monte Carlo simulation
# def simulate_game(home_team, away_team, timestamp, home_score, away_score, num_simulations=10000):
#     home_prior = prior_model.predict(np.array([[home_team['team_record'], home_team['recent_performance'], home_team['all_star'], home_team['offense_metric'], home_team['defense_metric']]]))[0]
#     away_prior = prior_model.predict(np.array([[away_team['team_record'], away_team['recent_performance'], away_team['all_star'], away_team['offense_metric'], away_team['defense_metric']]]))[0]
#     home_lead = home_score - away_score
    
#     home_wins = 0
#     away_wins = 0
    
#     for i in range(num_simulations):
#         # Simulate remaining game
#         home_simulation = np.random.normal(loc=home_prior, scale=1, size=1)
#         away_simulation = np.random.normal(loc=away_prior, scale=1, size=1)
        
#         time_remaining = 48 - timestamp
#         remaining_possessions = int(home_simulation + away_simulation + 2 * home_lead)
#         remaining_seconds = time_remaining * 60
        
#         home_points = 0
#         away_points = 0
        
#         while remaining_seconds > 0 and remaining_possessions > 0:
#             seconds_per_possession = remaining_seconds / remaining_possessions
#             time_used = np.random.exponential(scale=seconds_per_possession, size=1)
#             remaining_seconds -= time_used
#             remaining_possessions -= 1
            
#             home_efficiency = home_simulation / 100 * np.random.normal(loc=1, scale=0.1, size=1)
#             away_efficiency = away_simulation / 100 * np.random.normal(loc=1, scale=0.1, size=1)
            
#             home_points += np.random.poisson(home_efficiency * time_used / 60 * home_pace)
#             away_points += np.random.poisson(away_efficiency * time_used / 60 * away_pace)
        
#         if home_points > away_points:
#             home_wins += 1
#         else:
#             away_wins += 1
    
#     return {'home_win_prob': home_wins / num_simulations, 'away_win_prob': away_wins / num_simulations}